In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp tmlt

# Training Pipeline

> An API to create super fast training pipeline for machine learning models based on tabular or strucuture data

> It comes with model parallelism and cutting edge hyperparameter tuning techniques.

In [3]:
#hide
from nbdev.showdoc import *
from nbdev import *

In [4]:
# export
from tabular_ml_toolkit.dataframeloader import *
from tabular_ml_toolkit.preprocessor import *
from tabular_ml_toolkit.logger import *
from tabular_ml_toolkit.xgb_optuna_objective import *
from tabular_ml_toolkit.utility import *

In [5]:
# export
# hide
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
# for Optuna
import optuna
#for XGB
import xgboost
# for imbalance learn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

# for finding n_jobs in all sklearn estimators
from sklearn.utils import all_estimators
import inspect

# Just to compare fit times
import time

# for os specific settings
import gc
import os
from shutil import rmtree

#for clearning torch cache
import torch

In [6]:
# export

class TMLT:
    """
    Represent Tabular ML Toolkit class
    
    Attributes:\n
    dfl: A DataFrameLoader instance \n
    pp: A PreProcessor instance \n
    """

    def __init__(self):
        self.dfl = None
        self.pp = None
        self.transformer_type = None
        self.problem_type = None
        self.has_n_jobs = check_has_n_jobs()
        self.IDEAL_CPU_CORES = find_ideal_cpu_cores()
        
    
    def __str__(self):
        """Returns human readable string reprsentation"""
        attr_str = ("dfl, pp")
        return ("Training Pipeline object with attributes:"+attr_str)
    
    def __repr__(self):
        return self.__str__()
                
    ## All Core Methods ##
    
    # Main Method to create, load, preprocessed data based upon problem type
    def prepare_data(self, train_file_path:str,
                                  problem_type:str,
                                  idx_col:str, target:str,
                                  random_state:int,
                                  test_file_path:str=None,
                                  nrows=None):
        #set problem type
        self.problem_type = problem_type
        
        # call DataFrameLoader module
        self.dfl = DataFrameLoader().from_csv(
            train_file_path=train_file_path,
            test_file_path=test_file_path,
            idx_col=idx_col,
            target=target,
            random_state=random_state,
            nrows=nrows,
            problem_type=self.problem_type)
        
        # call PreProcessor module
        self.pp = PreProcessor().preprocess_all_cols(dataframeloader=self.dfl, problem_type=self.problem_type)

        # return tmlt
        gc.collect()
        return self
    
    #fit-tranform preprocessor
    def pp_fit_transform(self, X:object, y:object, X_test:object=None, resample_approach:str=None,
                         random_state:int=42):
        X_np = None
        X_test_np = None
        y_np = y
        #fit-transform
        if X is not None:
            X_np = self.pp.fit_transform(X)
            if X_test is not None:
                X_test_np = self.pp.transform(X_test)
        
        # resample can be done by many approaches, here we are using only 3
        if resample_approach is not None:
            if resample_approach == 'oversample':
                resample = SMOTE()
            elif resample_approach == 'undersample':
                resample = RandomUnderSampler(random_state=random_state)
            elif resample_approach == 'combine':
                resample = SMOTEENN(random_state=random_state)
            # now fit and tranform
            X_np, y_np = resample.fit_resample(X_np, y)
        
        return X_np, y_np, X_test_np
    
    # Force to update the dataframeloader in pipeline
    def update_dfl(self, X:object, y:object, X_test:object,
                   num_cols__imputer=SimpleImputer(strategy='median'),
                   num_cols__scaler=StandardScaler(),
                   cat_cols__imputer=SimpleImputer(strategy='constant'),
                   cat_cols__encoder=OneHotEncoder(handle_unknown='ignore')):
        
        # remove the old pipeline_cach directory
        if os.path.isdir("pipeline_cache_dir"):
            rmtree("pipeline_cache_dir")
        
        # regenerate dfl
        self.dfl = DataFrameLoader().regenerate_dfl(X,y,X_test)
        
        # change preprocessor
        self.pp = PreProcessor().preprocess_all_cols(self.dfl,
                                                     num_cols__imputer=num_cols__imputer,
                                                     num_cols__scaler=num_cols__scaler,
                                                     cat_cols__imputer=cat_cols__imputer,
                                                     cat_cols__encoder=cat_cols__encoder)
        gc.collect()
        return self
    
    
    # Force to update the preprocessor in pipeline
    def update_preprocessor(self,
                            num_cols__imputer=SimpleImputer(strategy='median'),
                            num_cols__scaler=StandardScaler(),
                            cat_cols__imputer=SimpleImputer(strategy='constant'),
                            cat_cols__encoder=OneHotEncoder(handle_unknown='ignore')):
        
        # remove the old pipeline_cach directory
        if os.path.isdir("pipeline_cache_dir"):
            rmtree("pipeline_cache_dir")
        
        # change preprocessor
        self.pp = PreProcessor().preprocess_all_cols(self.dfl,
                                                     num_cols__imputer=num_cols__imputer,
                                                     num_cols__scaler=num_cols__scaler,
                                                     cat_cols__imputer=cat_cols__imputer,
                                                     cat_cols__encoder=cat_cols__encoder)
        gc.collect()
        return self
    
    # cross validation
    def do_cross_validation(self, X:object, y:object, model:object, scoring:str, cv:int=5):
        """
        X, y
        model: takes any sklearn compatible model/estimator
        scoring: take str which are predefined here from https://scikit-learn.org/stable/modules/model_evaluation.html
        cv: takes int by default 5
        """
        scores = cross_val_score(
            estimator=model,
            X=X,
            y=y,
            scoring=scoring,
            cv=cv,
            n_jobs=self.IDEAL_CPU_CORES)
        
        # Multiply by -1 since sklearn calculates *negative* scoring for some of the metrics
        if "neg_" in scoring:
            scores = -1 * scores
        gc.collect()
        return scores
        
    # GridSearch
    def do_grid_search(self, X:object, y:object, param_grid:object, cv:int,
                       scoring:str, n_jobs=None):
        
        if n_jobs is None:
            n_jobs = self.IDEAL_CPU_CORES
        
        # create GridSeachCV instance
        grid_search = GridSearchCV(estimator=self.spl,
                                   param_grid=param_grid,
                                   cv=cv,
                                   scoring=scoring,
                                   n_jobs=n_jobs)
        # now call fit
        grid_search.fit(X, y)
        gc.collect()
        return grid_search
    
    # do oof k-fold train and predictions
    def do_oof_kfold_train_preds(self, X:object, y:object, n_splits:int, model:object, X_test:object=None,
                                 tabnet_params:dict=None, random_state=42):
        """
            This methods returns oof_preds and test_preds by doing kfold training on sklearn pipeline
            n_splits=5 by default, takes only int value
            random_sate=42, takes only int value

        """
        #TODO: Find better way without string matching
        if "tabnet" in str(model.__class__):
            #fetch problem type params
            _,_, eval_metric,_,oof_val_metric = fetch_tabnet_params_for_problem_type(self.problem_type)        
        
        elif "xgb" in str(model.__class__):
            #fetch problem type params
            _,_, eval_metric,_,oof_val_metric = fetch_xgb_params_for_problem_type(self.problem_type)
            
        else:
            #fetch problem type params
            _,_,oof_val_metric  = fetch_skl_params_for_problem_type(self.problem_type)
        
        #create stratified K Folds instance
        kfold = StratifiedKFold(n_splits=n_splits,
                             random_state=random_state,
                             shuffle=True)

        # for oof preds
        oof_preds = np.zeros(X.shape[0])

        # check whether test dataset exist before test preds
        oof_test_preds = None
        if X_test is not None:
            oof_test_preds = np.zeros(X_test.shape[0])

        # k-fold training and predictions for oof predictions
        oof_model_results_mean = 0
        n=0
        for train_idx, valid_idx in kfold.split(X, y):
            if isinstance(X, np.ndarray):
                # create NUMPY based X_train, X_valid, y_train, y_valid
                # fix the stratification problem
                #train_idx,valid_idx = clip_splits(train_idx,valid_idx,self.dfl.X_full)
                X_train , X_valid, y_train, y_valid = X[train_idx], X[valid_idx], y[train_idx], y[valid_idx]
            else:
                # create PANDAS based X_train, X_valid, y_train, y_valid
                # fix the stratification problem
                #train_idx,valid_idx = clip_splits(train_idx,valid_idx,self.dfl.X_full)
                X_train , X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
                
            #TRAINING
            logger.info(f"Training Started!")
                        #should be better way without string matching
            if "tabnet" in str(model.__class__):
                #change for tabnet
                if  tabnet_params:
                    model.fit(X_train, y_train,
                              eval_set=[(X_valid, y_valid)],
                              eval_metric=[eval_metric],
                             **tabnet_params)        

            elif "xgb" in str(model.__class__):
                #change for xgb
                model.fit(X_train, y_train,
                          eval_set=[(X_train, y_train), (X_valid, y_valid)],
                          eval_metric=eval_metric,
                         verbose=False)

            else:
                #standard sklearn fit
                model.fit(X_train, y_train)
            logger.info(f"Training Finished!")

            #VAL PREDICTIONS
            # getting either hyperplane distance or probablities from predictions
            if "svm" in str(model.__class__):
                #logger.info(f"Predicting Valid Decision!")
                preds = model.decision_function(X_valid)
                oof_preds[valid_idx] = preds
            else:
                #logger.info(f"Predicting Valid Proba!")
                if "classification" in self.problem_type:
                    preds = model.predict_proba(X_valid)
                    oof_preds[valid_idx] = preds[:,1]
                else:
                    preds = model.predict(X_valid)
                    oof_preds[valid_idx] = preds
                    

            #METRICS
            # Get metric results for each fold
            oof_metric_results = oof_val_metric(y_valid, preds)
            logger.info(f"fold: {n+1} OOF {str(oof_val_metric.__name__)}: {oof_metric_results}!")
            
            # for mean score
            oof_model_results_mean += (oof_metric_results / kfold.n_splits)


            #TEST PREDICTIONS
            # appending mean test data predictions
            if X_test is not None:
                if "svm" in str(model.__class__):
                    oof_test_preds += model.decision_function(X_test) / kfold.n_splits
                else:
                    if "classification" in self.problem_type:
                        oof_test_preds += model.predict_proba(X_test)[:,1] / kfold.n_splits
                    else:
                        oof_test_preds += model.predict(X_test) / kfold.n_splits
            else:
                logger.warn(f"Trying to do OOF Test Predictions but No Test Dataset Provided!")

            # In order to better GC, del X_train, X_valid, y_train, y_valid df after each fold is done,
            # they will recreate again next time k-fold is called
            del [X_train, X_valid, y_train, y_valid]
            gc.collect()
            torch.cuda.empty_cache()

            # increment fold counter label
            n += 1

        #oof_model_auc_mean = (oof_model_auc / kfold.n_splits)
        logger.info(f"Mean OOF {str(oof_val_metric.__name__)}: {oof_model_results_mean}!")
        del [kfold, model]
        gc.collect()
        torch.cuda.empty_cache()

        return oof_preds, oof_test_preds
    
    
    # do k-fold training
    def do_kfold_training(self, X:object, y:object, n_splits:int, model:object, kfold_metric:object,
                          eval_metric:str, X_test:object=None, tabnet_params:dict=None, random_state=42):
        
        """
            This methods returns kfold_metrics_results and test_preds by doing kfold training
            n_splits=5 by default, takes only int value
            random_sate=42, takes only int value

        """ 
        
        #logger.info(f" model class:{model.__class__}")
        
        #create stratified K Folds instance
        kfold = StratifiedKFold(n_splits=n_splits,
                             random_state=random_state,
                             shuffle=True)    
        
        # check for test dataset before prediction
        test_preds = None
        if X_test is not None:
            test_preds = np.zeros(X_test.shape[0])
        
        # list contains metrics results for each fold
        kfold_metrics_results = []
        n=0
        for train_idx, valid_idx in kfold.split(X, y):
            if isinstance(X, np.ndarray):
                # create NUMPY based X_train, X_valid, y_train, y_valid
                X_train , X_valid, y_train, y_valid = X[train_idx], X[valid_idx], y[train_idx], y[valid_idx]
            else:
                # create PANDAS based X_train, X_valid, y_train, y_valid
                X_train , X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
            
            #TRAINING
            logger.info(f"Training Started!")
            #should be better way without string matching
            if "tabnet" in str(model.__class__):
                #change for tabnet
                model.fit(X_train, y_train,
                          eval_set=[(X_valid, y_valid)],
                          eval_metric=[eval_metric],
                          **tabnet_params)        

            elif "xgb" in str(model.__class__):
                #change for xgb
                model.fit(X_train, y_train,
                          eval_set=[(X_valid, y_valid)],
                          eval_metric=eval_metric,
                         verbose=False)

            else:
                #standard sklearn fit
                model.fit(X_train, y_train)
            
            logger.info(f"Training Finished!")
            
            #TO-DO: Merge OOF_KFold and KFold methods at this level
            metric_result = {}
            preds = None
            metric = kfold_metric
            
            #VAL PREDICTIONS
            if "svm" in str(model.__class__):
                logger.info("Predicting Val Decision Function!")
                preds = model.decision_function(X_valid)
            else:
                if needs_predict_proba(metric):
                    logger.info("Predicting Val Probablities!")
                    preds = model.predict_proba(X_valid)[:, 1]
                else:
                    logger.info("Predicting Val Score!")
                    preds = model.predict(X_valid)
            
            #METRIC CALCULATION
            metric_result[str(metric.__name__)] = metric(y_valid, preds)

            #now show value of all the given metrics
            for metric_name, metric_value in metric_result.items():
                logger.info(f"fold: {n+1} {metric_name} : {metric_value}")
            
            #now append each kfold metric_result dict to list
            kfold_metrics_results.append(metric_result)
            
            ##TEST PREDICTIONS
            if X_test is not None:
                logger.info("Predicting Test Scores!")
                test_preds += model.predict(X_test) / kfold.n_splits
            else:
                logger.warn(f"Trying to do Test Predictions but No Test Dataset Provided!")

            # In order to better GC, del X_train, X_valid, y_train, y_valid df after each fold is done,
            # they will recreate again next time k-fold is called
            del [X_train, X_valid, y_train, y_valid]
            gc.collect()
            torch.cuda.empty_cache()
            
            # increment fold counter label
            n += 1
        
        # inverse tranform labels predicted in test preds
        test_preds = self.pp.target_cols__encoder.inverse_transform(test_preds)
        # (test_proba.argmax(axis=1))
        #logger.info(f"kfold_metrics_results: {kfold_metrics_results} ")
        mean_metrics_results = kfold_dict_mean(kfold_metrics_results)
        logger.info(f" Mean {str(metric.__name__)} from all Folds are: {mean_metrics_results}")
        # for explicit memory clean
        del [kfold, model]
        gc.collect()
        torch.cuda.empty_cache()
        return mean_metrics_results, test_preds
    
    # Do optuna bases study optimization for hyperparmaeter search
    def do_xgb_optuna_optimization(self, X_train_np, y_train_np, X_valid_np, y_valid_np, optuna_db_path:str,
                                   use_gpu=False, opt_trials=100, opt_timeout=360, verbose=False):
        """
            This methods returns and do optuna bases study optimization for hyperparmaeter search
            optuna_db_path is output directory you want to use for storing sql db used for optuna
            use_gpu=False by default, make it True if running on gpu machine
            opt_trials=100 by default, change it based upon need
            opt_timeout=360 by default, timeout value in seconds

        """       
            
        # get params based on problem type
        xgb_model_type, val_preds_metrics, eval_metric, direction, _ = fetch_xgb_params_for_problem_type(self.problem_type)
        
        # Load the dataset in advance for reusing it each trial execution.
        objective = XGB_Optuna_Objective(X_train_np, y_train_np, X_valid_np, y_valid_np,
                                         val_preds_metrics=val_preds_metrics,
                                         xgb_model_type=xgb_model_type, xgb_eval_metric=eval_metric,
                                         use_gpu=use_gpu, verbose=verbose)
        # create sql db in optuna db path
        db_path = os.path.join(optuna_db_path, "params.db")
        
        # now create study
        logger.info(f"Optimization Direction is: {direction}")
        study = optuna.create_study(
            direction=direction,
            study_name="tmlt_autoxgb",
            storage=f"sqlite:///{db_path}",
            load_if_exists=True,
        )
        study.optimize(objective, n_trials=opt_trials, timeout=opt_timeout)
        gc.collect()
        return study

    
    # helper methods for users before updating preprocessor in pipeline
    def get_preprocessor_best_params_from_grid_search(self, grid_search_object:object):
        pp_best_params = {}
        for k in grid_search_object.best_params_:
            #print(k)
            if 'preprocessor' in k:
                key = k.split('__')[1] + "__" + k.split('__')[2] 
                pp_best_params[key] = grid_search_object.best_params_[k]
        return pp_best_params
    
    # helper methods for users before updating model in pipeline
    def get_model_best_params_from_grid_search(self, grid_search_object:object):
        model_best_params = {}
        for k in grid_search_object.best_params_:
            #print(k)
            if 'model' in k:
                key = k.split('__')[1]
                model_best_params[key] = grid_search_object.best_params_[k]
        return model_best_params

In [7]:
show_doc(TMLT)

<h2 id="TMLT" class="doc_header"><code>class</code> <code>TMLT</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>TMLT</code>()

Represent Tabular ML Toolkit class

Attributes:

dfl: A DataFrameLoader instance 

pp: A PreProcessor instance 

In [8]:
show_doc(TMLT.prepare_data)

<h4 id="TMLT.prepare_data" class="doc_header"><code>TMLT.prepare_data</code><a href="__main__.py#L32" class="source_link" style="float:right">[source]</a></h4>

> <code>TMLT.prepare_data</code>(**`train_file_path`**:`str`, **`problem_type`**:`str`, **`idx_col`**:`str`, **`target`**:`str`, **`random_state`**:`int`, **`test_file_path`**:`str`=*`None`*, **`nrows`**=*`None`*)



In [9]:
show_doc(TMLT.do_xgb_optuna_optimization)

<h4 id="TMLT.do_xgb_optuna_optimization" class="doc_header"><code>TMLT.do_xgb_optuna_optimization</code><a href="__main__.py#L409" class="source_link" style="float:right">[source]</a></h4>

> <code>TMLT.do_xgb_optuna_optimization</code>(**`X_train_np`**, **`y_train_np`**, **`X_valid_np`**, **`y_valid_np`**, **`optuna_db_path`**:`str`, **`use_gpu`**=*`False`*, **`opt_trials`**=*`100`*, **`opt_timeout`**=*`360`*, **`verbose`**=*`False`*)

This methods returns and do optuna bases study optimization for hyperparmaeter search
optuna_db_path is output directory you want to use for storing sql db used for optuna
use_gpu=False by default, make it True if running on gpu machine
opt_trials=100 by default, change it based upon need
opt_timeout=360 by default, timeout value in seconds

In [10]:
# hide
# run the script to build 

from nbdev.export import notebook2script; notebook2script()

Converted 00_dataframeloader.ipynb.
Converted 01_preprocessor.ipynb.
Converted 02_tmlt.ipynb.
Converted 04_xgb_optuna_objective.ipynb.
Converted Kaggle_TPS_Dec_Meta_Clean.ipynb.
Converted Kaggle_TPS_Dec_TabNet.ipynb.
Converted Kaggle_TPS_Dec_Tutorial-Meta(Tabnet+XGB).ipynb.
Converted Kaggle_TPS_Dec_Tutorial-Meta.ipynb.
Converted Kaggle_TPS_Dec_Tutorial_XGB.ipynb.
Converted Kaggle_TPS_Nov_Tutorial.ipynb.
Converted index.ipynb.
Converted logger.ipynb.
Converted utility.ipynb.
Converted xgb_tabular_ml_toolkit.ipynb.


<!-- 
run script to update all files

make pypi

reconnect and reinstall tmlt on google colab, restart colab
use ensembled oof model code from this notebook and add it to google colab
make sure to comment out nrows=4000
let it run and wait
get submissions and download it
upload it to kaggle to see results
 -->